In [1]:
import tensorflow as tf

# Load your trained model
model_path = 'densenet.h5'  # Update with your model's path
model = tf.keras.models.load_model(model_path)


In [2]:
import cv2
import os

def extract_frames(video_path, output_dir):
    print("Extracting Frames")
    print(video_path,output_dir)
    # Create a directory for frames
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open the video
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Save frame as image
        cv2.imwrite(os.path.join(output_dir, f'frame_{count:04d}.png'), frame)
        count += 1
    cap.release()
    print(f"Extracted {count} frames to {output_dir}")

# video_path = 'input_output/Input/car_a.mp4'  # Update with your video path
# output_dir = 'input_output/Input/car_a/frames/'    # Update with your desired output directory
# extract_frames(video_path, output_dir)


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

IMG_HEIGHT, IMG_WIDTH = 64,64

def predict_frames(frames_dir):
    print("Pridicting Frames")
    predictions = []
    frame_files = sorted(os.listdir(frames_dir))
    for frame_file in frame_files:
        frame_path = os.path.join(frames_dir, frame_file)
        img = load_img(frame_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = img_to_array(img) / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        prediction = model.predict(img_array)
        predictions.append(prediction)
    print("Frames Predicted")
    return predictions

# predictions = predict_frames(output_dir)


In [4]:
from scipy.stats import entropy

def score_frames(predictions):
    print("Scoring Frames")
    scores = []
    for i, pred in enumerate(predictions):
        # Example scoring based on entropy of class probabilities
        score = -np.sum(pred * np.log(pred + 1e-10))  # Entropy
        # Additional scoring mechanisms can be added here
        scores.append(score)
    print("Frames Scored")
    return scores

# scores = score_frames(predictions)
# print(scores)


In [5]:
def rank_shots(scores):
    print("Ranking Frames")
    ranked_indices = np.argsort(scores)[::-1]  # Sort in descending order
    print("Frames Ranked")
    return ranked_indices
# ranked_indices = rank_shots(scores)
# print(ranked_indices)


In [6]:
def summarize_video(frames_dir, ranked_indices, output_video_path, num_keyframes=50, frame_rate=24):
    print("Summarizing Frames")
    selected_frames = ranked_indices[:num_keyframes]
    keyframe_paths = [os.path.join(frames_dir, f'frame_{index:04d}.png') for index in selected_frames][::-1]
    
    # Check the first frame to get dimensions
    first_frame = cv2.imread(keyframe_paths[0])
    if first_frame is None:
        print(f"Error loading image: {keyframe_paths[0]}")
        return

    height, width, layers = first_frame.shape

    # Initialize VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))
    
    for keyframe_path in keyframe_paths:
        print(f"Attempting to load: {keyframe_path}")  # Debug output
        img = cv2.imread(keyframe_path)
        if img is None:
            print(f"Error loading image: {keyframe_path}")
        else:
            # cv2.imshow("Keyframe", img)
            img_resized = cv2.resize(img, (width, height))
            out.write(img_resized)
            # cv2.waitKey(500)
    out.release()
    print(f"Summarized video saved to {output_video_path}")
    # cv2.destroyAllWindows()

# summarize_video(output_dir, ranked_indices, "input_output/output/Arson002_x264/car_a.mp4")


In [7]:
def summarize_from_path(video_path, output_dir):
    extract_frames(video_path, output_dir)